In [1]:
import json
import random

In [2]:
lang = "en"
diff = "easy"
length = 4

vocabs_pool: set[str] = set()
exclusive_pool: set[str] = set()
alphabets: set[str] = set()

with open(rf"vocabs\{lang}\{lang}-len{length}-{diff}.json") as file:
    json_data = json.load(file)
    for d in json_data:
        vocabs_pool.add(d)
        exclusive_pool.add(d)
        alphabets = alphabets.union(set(d))

# with open(r"vocabs\{lang}\{lang}-len{length}-medium.json") as file:
#     json_data = json.load(file)
#     for d in json_data:
#         exclusive_pool.remove(d)

num_vocabs = len(vocabs_pool)
print(len(alphabets), "".join(sorted(alphabets)))
print(f"{len(vocabs_pool)=}")
print(f"{len(exclusive_pool)=}")

26 abcdefghijklmnopqrstuvwxyz
len(vocabs_pool)=402
len(exclusive_pool)=402


In [3]:
def change_character(vocab: str) -> set[str]:
    new_vocabs = set()
    for i, c in enumerate(vocab):
        for a in alphabets:
            if a != c:
                new_vocab = vocab[:i] + a + vocab[i + 1 :]
                new_vocabs.add(new_vocab)

    return new_vocabs


print(change_character("abc"))

{'ibc', 'lbc', 'abj', 'abs', 'abe', 'abr', 'nbc', 'hbc', 'qbc', 'anc', 'abh', 'atc', 'mbc', 'abo', 'xbc', 'ubc', 'abz', 'ajc', 'abn', 'aic', 'kbc', 'abx', 'aqc', 'apc', 'avc', 'tbc', 'asc', 'wbc', 'azc', 'abt', 'ahc', 'acc', 'fbc', 'abd', 'abu', 'awc', 'abp', 'pbc', 'abq', 'abw', 'aby', 'cbc', 'abf', 'abk', 'abg', 'gbc', 'vbc', 'auc', 'dbc', 'jbc', 'ebc', 'abm', 'bbc', 'amc', 'abv', 'agc', 'afc', 'abl', 'obc', 'arc', 'abi', 'sbc', 'rbc', 'ybc', 'akc', 'aec', 'aoc', 'ayc', 'abb', 'aac', 'zbc', 'adc', 'alc', 'aba', 'axc'}


In [4]:
def character_difference(from_: str, to: str) -> int:
    count = 0
    for c1, c2 in zip(from_, to):
        if c1 != c2:
            count += 1
    return count

In [5]:
def distance(
    from_: str, to: str, max_distance: int | None = None
) -> tuple[int, list[str]]:
    vocab_path = {from_: ""}
    queue = [(from_, 0)]
    while queue:
        (cur_vocab, cur_distance) = queue.pop(0)
        if cur_vocab == to:
            path = [cur_vocab]
            while path[0] != from_:
                path.insert(0, vocab_path[path[0]])
            return (cur_distance, path)

        if cur_distance == max_distance:
            continue

        for new_vocab in change_character(cur_vocab):
            if (new_vocab not in vocab_path) and (new_vocab in vocabs_pool):
                vocab_path[new_vocab] = cur_vocab
                queue.append((new_vocab, cur_distance + 1))

    return (-1, [])

In [10]:
print(distance("admin", "admit"))
print(distance("line", "firm"))
print(distance("war", "him"))
print(distance("センセイ", "モンダイ"))

(-1, [])
(3, ['line', 'fine', 'fire', 'firm'])
(-1, [])
(-1, [])


In [7]:
# easy: {"max_distance": 4, "min_distance": 3, "strict": False}

max_distance = 4
strict = False
min_distance = 3

count = 0
count_exclusive = 0
starter = set()
involved = set()
pairs: list[tuple[str, str]] = []

for i, v1 in enumerate(vocabs_pool):
    for j, v2 in enumerate(vocabs_pool):
        dist = distance(v1, v2, max_distance=max_distance)
        if strict:
            difference = character_difference(v1, v2)
            flag = dist[0] > difference and dist[0] >= min_distance
        else:
            flag = dist[0] >= min_distance

        if flag:
            count += 1
            starter.add(v1)
            involved = involved.union(set(dist[1]))
            # print(dist)

            if any(v in exclusive_pool for v in dist[1]):
                count_exclusive += 1
                pairs.append((v1, v2))

    if i % (num_vocabs // 10) == 0:
        print(f'checking "{v1}"| {i}/{num_vocabs}')

total_combinations = num_vocabs * (num_vocabs - 1)

pair_rate = count / total_combinations
print(f"pair rate: {count}/{total_combinations} = {pair_rate:.2%}")

exclusive_rate = count_exclusive / total_combinations
print(
    f"exclusive pair rate: {count_exclusive}/{total_combinations} = {exclusive_rate:.2%}"
)

starter_rate = len(starter) / num_vocabs
print(f"starter: {len(starter)}/{num_vocabs} = {starter_rate:.2%}")

participation_rate = len(involved) / num_vocabs
print(f"participation: {len(involved)}/{num_vocabs} = {participation_rate:.2%}")

checking "tour"| 0/402


checking "soft"| 40/402
checking "wide"| 80/402
checking "hour"| 120/402
checking "inch"| 160/402
checking "hand"| 200/402
checking "late"| 240/402
checking "past"| 280/402
checking "send"| 320/402
checking "fast"| 360/402
checking "less"| 400/402
pair rate: 4148/161202 = 2.57%
exclusive pair rate: 4148/161202 = 2.57%
starter: 239/402 = 59.45%
participation: 241/402 = 59.95%


In [8]:
random.seed(0)
pairs_shuffled = pairs[:]
random.shuffle(pairs_shuffled)
with open(rf"vocabs\{lang}\{lang}-len{length}-pairs-{diff}.json", "w") as f:
    json.dump(pairs_shuffled, f)

for i in range(5):
    pair = pairs_shuffled[i]
    dist = distance(pair[0], pair[1])
    print(dist)

(3, ['firm', 'fire', 'fine', 'line'])
(3, ['more', 'mode', 'made', 'male'])
(3, ['link', 'line', 'like', 'lake'])
(3, ['fact', 'fast', 'east', 'easy'])
(4, ['care', 'case', 'cast', 'cost', 'host'])


In [9]:
def create_distance_graph():
    distance_graph: dict[tuple[str, str], tuple[int, list[str]]] = {}

    def calculate_distance(from_: str, to: str):
        if (from_, to) in distance_graph:
            return

        min_distance = -1
        min_path = []
        for new_vocab in change_character(from_):
            if new_vocab in vocabs_pool:
                calculate_distance(new_vocab, to)
                distance, path = distance_graph[(new_vocab, to)]
                if min_distance == -1 or 0 <= distance < min_distance:
                    min_distance = distance + 1
                    min_path = [from_] + path
        distance_graph[(from_, to)] = (min_distance, min_path)

    for v1 in vocabs_pool:
        for v2 in vocabs_pool:
            calculate_distance(v1, v2)
    return distance_graph


# distance_graph = create_distance_graph()
# print(distance_graph)